In [1]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from keras.layers import Input, Convolution1D, MaxPooling1D, UpSampling1D, Masking
from keras.models import Layer, Model

from keras.optimizers import SGD

import keras.backend as K
import numpy as np
seed=6
np.random.seed(seed)

import os
os.chdir('..')
from evolutron.networks import Deconvolution1D, Unpooling1D


Using Theano backend.


In [2]:
aa_length=15
filter_length=3
nb_filter=2
alphabet=4

In [3]:
conv = Convolution1D(nb_filter, filter_length,
                               init='glorot_normal',
                               activation='relu',
                               border_mode='same',
                               name='Conv1')

maxpool = MaxPooling1D(pool_length=aa_length)

In [4]:
unpool = Unpooling1D(maxpool)
mask = Masking(mask_value=0.0)
deconv = Deconvolution1D(conv)

In [5]:
inp = Input(shape=(aa_length, alphabet), name='aa_seq')
x_data = np.random.random((10, 10, 4))
x_data = np.pad(x_data, ((0,0), (0,5), (0,0)), 'constant', constant_values=0)
x_data = x_data.astype(np.float32)

In [6]:
x_m = mask(inp)
x_c = conv(x_m)
x_p = maxpool(x_c)
x_u = unpool(x_p)
x_d = deconv(x_u)

In [7]:
coder = Model(input=inp, output=x_d)

In [8]:
coder.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
aa_seq (InputLayer)              (None, 15, 4)         0                                            
____________________________________________________________________________________________________
masking_1 (Masking)              (None, 15, 4)         0           aa_seq[0][0]                     
____________________________________________________________________________________________________
Conv1 (Convolution1D)            (None, 15, 2)         26          masking_1[0][0]                  
____________________________________________________________________________________________________
maxpooling1d_1 (MaxPooling1D)    (None, 1, 2)          0           Conv1[0][0]                      
___________________________________________________________________________________________

In [9]:
coder.compile(optimizer='adam', loss='mse')

In [10]:
coder.fit(x_data, x_data, nb_epoch=100)

Epoch 1/100
10/10 [==============================] - 0s - loss: 0.3103
Epoch 2/100
10/10 [==============================] - 0s - loss: 0.3075
Epoch 3/100
10/10 [==============================] - 0s - loss: 0.3048
Epoch 4/100
10/10 [==============================] - 0s - loss: 0.3021
Epoch 5/100
10/10 [==============================] - 0s - loss: 0.2994
Epoch 6/100
10/10 [==============================] - 0s - loss: 0.2968
Epoch 7/100
10/10 [==============================] - 0s - loss: 0.2942
Epoch 8/100
10/10 [==============================] - 0s - loss: 0.2916
Epoch 9/100
10/10 [==============================] - 0s - loss: 0.2890
Epoch 10/100
10/10 [==============================] - 0s - loss: 0.2864
Epoch 11/100
10/10 [==============================] - 0s - loss: 0.2839
Epoch 12/100
10/10 [==============================] - 0s - loss: 0.2814
Epoch 13/100
10/10 [==============================] - 0s - loss: 0.2789
Epoch 14/100
10/10 [==============================] - 0s - loss: 0.2764
E

In [11]:
print(x_data.squeeze()[0])
print(coder.predict(x_data).squeeze()[0])
print(coder.evaluate(x_data, x_data))

[[ 0.89286017  0.33197981  0.8212291   0.04169663]
 [ 0.10765668  0.59505206  0.52981734  0.41880742]
 [ 0.33540785  0.62251943  0.43814144  0.7358821 ]
 [ 0.51803643  0.57885861  0.64535511  0.99022424]
 [ 0.81985819  0.41320094  0.87626767  0.82375944]
 [ 0.05447451  0.71863723  0.80217057  0.73640662]
 [ 0.70913178  0.54093683  0.12482417  0.95764732]
 [ 0.4032563   0.21695116  0.71727586  0.99420744]
 [ 0.25561404  0.67130941  0.59900594  0.71733212]
 [ 0.93734956  0.35180977  0.2536341   0.4024725 ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
[[ 0.35977209  0.29158431  0.39087862  0.63980585]
 [ 0.10104732  0.31302565  0.55286384  0.59580714]
 [ 0.10104732  0.31302565  0.55286384  0.59580714]
 [ 0.10104732  0.31302565  0.55286384  0.59580714]
 [ 0.10104732  0.31302565  0.5

In [12]:
np.mean(np.square(x_data-coder.predict(x_data)), dtype=np.float32)

0.090429239

In [13]:
f_c = K.function([inp], x_c)
f_p = K.function([inp], x_p)
f_u = K.function([inp], x_u)
f_m = K.function([inp], x_m)
f_d = K.function([inp], x_d)

In [14]:
v_c = f_c([np.expand_dims(x_data[0],0)])
v_p = f_p([np.expand_dims(x_data[0],0)])
v_u = f_u([np.expand_dims(x_data[0],0)])
v_m = f_m([np.expand_dims(x_data[0],0)])
v_d = f_d([np.expand_dims(x_data[0],0)])

In [15]:
f_d = K.function([inp], x_d)
v_d = f_d([x_data])
v_d.shape

(10, 15, 4)

import theano
import theano.tensor as ten
import lasagne

las_inp_var = ten.tensor3('aa_seq', dtype=theano.config.floatX)
las_target = ten.tensor3('aa_rec_seq', dtype=theano.config.floatX)

las_inp = lasagne.layers.InputLayer(input_var=las_inp_var,
                                    shape=(None,alphabet,aa_length),
                                    name='Input')
las_conv = lasagne.layers.Conv1DLayer(las_inp,
                                     num_filters=nb_filter,
                                     filter_size=filter_length,
                                     flip_filters=False,
                                     nonlinearity=lasagne.nonlinearities.rectify,
                                     W=lasagne.init.GlorotUniform('relu'),
                                     stride=1,
                                     pad='full',
                                     name='Conv1')

las_maxpool = lasagne.layers.MaxPool1DLayer(las_conv, pool_size=aa_length,
                                          name='MaxPool')
las_unpool = lasagne.layers.InverseLayer(las_maxpool, las_maxpool, name='inv_pool')

las_deconv= lasagne.layers.InverseLayer(las_unpool, las_conv, name='inv_conv')

prediction = lasagne.layers.get_output(las_deconv)
loss = lasagne.objectives.squared_error(prediction, las_target)
loss = loss.mean()

test_prediction = lasagne.layers.get_output(las_deconv, deterministic=True)
test_loss = lasagne.objectives.squared_error(test_prediction, las_target)
test_loss = test_loss.mean()

params = lasagne.layers.get_all_params(las_deconv, trainable=True)
updates = lasagne.updates.adam(loss, params)

train_fn = theano.function(inputs=[las_inp_var, las_target],
                           outputs=[loss],
                           updates=updates,
                           allow_input_downcast=False)

test_fn = theano.function(inputs=[las_inp_var, las_target],
                          outputs=[test_loss],
                          allow_input_downcast=False)

pred_fn = theano.function(inputs=[las_inp_var],
                          outputs=[test_prediction],
                          allow_input_downcast=False)

lf_c = theano.function(inputs=[las_inp_var],
                       outputs=[lasagne.layers.get_output(las_conv)],
                       allow_input_downcast=False)

lf_p = theano.function(inputs=[las_inp_var],
                       outputs=[lasagne.layers.get_output(las_maxpool)],
                       allow_input_downcast=False)
lf_u = theano.function(inputs=[las_inp_var],
                       outputs=[lasagne.layers.get_output(las_unpool)],
                       allow_input_downcast=False)
lf_d = theano.function(inputs=[las_inp_var],
                       outputs=[lasagne.layers.get_output(las_deconv)],
                       allow_input_downcast=False)

for i in range(0,100):
    print(train_fn(x_data.transpose((0,2,1)), x_data.transpose((0,2,1)))[0])

lv_c = lf_c([x_data[0].T])[0].squeeze()
lv_p = lf_p([x_data[0].T])[0].squeeze()
lv_u = lf_u([x_data[0].T])[0].squeeze()
lv_d = lf_d([x_data[0].T])[0].squeeze()

In [16]:
print(v_c)

[[[ -3.72529030e-09   7.81332314e-01]
  [ -3.72529030e-09   5.53094864e-01]
  [ -3.72529030e-09   8.66643071e-01]
  [ -3.72529030e-09   9.01467562e-01]
  [ -3.72529030e-09   1.03664315e+00]
  [ -3.72529030e-09   8.03544819e-01]
  [  1.11758709e-08   9.14665043e-01]
  [ -3.72529030e-09   7.25457609e-01]
  [ -3.72529030e-09   6.37077332e-01]
  [ -3.72529030e-09   5.16717672e-01]
  [ -3.72529030e-09   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00]]]


In [17]:
print(v_p)

[[[  1.11758709e-08   1.03664315e+00]]]


In [18]:
print(v_u)

[[[  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]
  [  1.11758709e-08   1.03664315e+00]]]


In [19]:
print(v_d)
print(x_data[0])

[[[ 0.35977209  0.29158431  0.39087862  0.63980585]
  [ 0.10104732  0.31302565  0.55286384  0.59580714]
  [ 0.10104732  0.31302565  0.55286384  0.59580714]
  [ 0.10104732  0.31302565  0.55286384  0.59580714]
  [ 0.10104732  0.31302565  0.55286384  0.59580714]
  [ 0.10104732  0.31302565  0.55286384  0.59580714]
  [ 0.10104732  0.31302565  0.55286384  0.59580714]
  [ 0.10104732  0.31302565  0.55286384  0.59580714]
  [ 0.10104732  0.31302565  0.55286384  0.59580714]
  [ 0.10104732  0.31302565  0.55286384  0.59580714]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 0.35982227  0.29162312  0.39095309  0.63991541]
  [ 0.10104913  0.31306851  0.55296862  0.59590846]
  [ 0.10104913  0.31306851  0.55296862  0.59590846]
  [ 0.10104913  0.31306851  0.55296862  0.59590846]
  [ 0.1010